## Introduction to the Keras Tuner 

### Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:

<ul>
    <li>Model hyperparameters which influence model selection such as the number and width of hidden layers</li>
    <li>Algorithm hyperparameters which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier</li>
</ul>

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

In [2]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [3]:
!pip install -q -U keras-tuner

In [4]:
import keras_tuner as kt

### Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion MNIST dataset</a>.

Load the data.

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

<ul>
    <li>By using a model builder function</li>
    <li>By subclassing the <code>HyperModel</code> class of the Keras Tuner API</li>
</ul>

You can also use two pre-defined <a href="https://keras.io/api/keras_tuner/hypermodels/">HyperModel</a> classes - <a href="https://keras.io/api/keras_tuner/hypermodels/hyper_xception/">HyperXception</a> and <a href="https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/">HyperResNet</a> for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

### Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - <code>RandomSearch</code>, <code>Hyperband</code>, <code>BayesianOptimization</code>, and <code>Sklearn</code>. In this tutorial, you use the <a href="https://arxiv.org/pdf/1603.06560.pdf">Hyperband</a> tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to <code>optimize</code> and the maximum number of epochs to train (<code>max_epochs</code>).

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

2025-04-01 03:54:45.575252: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub><code>factor</code></sub>(<code>max_epochs</code>) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for <code>tf.keras.model.fit</code> in addition to the callback above.

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 21 Complete [00h 00m 14s]
val_accuracy: 0.8817499876022339

Best val_accuracy So Far: 0.8915833234786987
Total elapsed time: 00h 03m 13s

Search: Running Trial #22

Value             |Best Value So Far |Hyperparameter
448               |448               |units
0.01              |0.001             |learning_rate
4                 |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/4
  22/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3349 - loss: 4.3895     

### Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

To finish this tutorial, evaluate the hypermodel on the test data.

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

The <code>my_dir/intro_to_kt</code> directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional <code>overwrite=True</code> argument while instantiating the tuner.

### Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

<ul>
    <li><a href="https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html?_gl=1*1mthrdj*_ga*MTUyODQ2MzYwMS4xNjY5NzgyMDI2*_ga_W0YLR4190T*MTc0MzM5MTI3MS41Mi4xLjE3NDMzOTEyNzMuMC4wLjA.">Keras Tuner on the TensorFlow blog</a></li>
    <li><a href="https://keras-team.github.io/keras-tuner/">Keras Tuner website</a></li>
</ul>

Also check out the <a href="https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams">HParams Dashboard</a> in TensorBoard to interactively tune your model hyperparameters.